### Create Character Vector Database

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import CohereEmbeddings
import json
import random
from langchain.vectorstores import Chroma

# Set the game name and character name
game_name = 'Cyberpunk_2077'
character_name = 'Johnny_Silverhand'

# Load the file
with open(f'{game_name}/characters/{character_name}/character_knowledge.txt', errors='ignore') as f:
    public_info = f.read()

# Create Text Splitter, Chunk size is number of characters in each item of the list 
# While chunk overlap is how many characters from the previous item should be repeated in the next item 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap  = 0,
    length_function = len,
    separators = ['\n\n\n', '\n\n', '\n', '.', ',', ' ']
)

# Split the text 
public_info_list = text_splitter.split_text(public_info)

# Randomly select an API key
selected_key = json.load(open('apikeys.json', 'r'))['api_keys'][random.randint(
    0, len(json.load(open('apikeys.json', 'r'))['api_keys'])-1)]

# Initialise the embedding model
embeddings = CohereEmbeddings(cohere_api_key=selected_key)

# Path must be inside the character folder in the game folder with the name vectordb
persist_directory = f'{game_name}/characters/{character_name}/vectordb'

# Create the Vector Database
vectordb = Chroma.from_texts(public_info_list, embeddings, persist_directory=persist_directory)
vectordb.persist()

### Test the Vector Database

In [2]:
# search for a topic and see if you get relevant information, k is for number of similar items to return
docs = vectordb.similarity_search("Rogue", k = 3)
similar_story_string = "\n".join(doc.page_content for doc in docs)
print(similar_story_string)

Rogue was a rebellious teen, knew stiff hierarchy of gang life was not for her and set out on her own, to become a solo. She was a new and successful mercenary in Night City. Rogue also began a relationship with hit rockerboy sensation Johnny Silverhand. Rogue on occasion asked Johnny if they could visit a drive-in theatre on a date, which went largely ignored.

Her relationship with Johnny ended at some point in 2011,[5] after Rogue found out about Johnny cheating on her.[6]
Rogue's relationship with Johnny ended at some point in 2011,[12] after she found out about Johnny cheating on her.[13]

After giving a concert at The Hammer on August 4th in 2013, Johnny went on a walk with Alt Cunningham, his former girlfriend, but were ambushed by thugs hired by Arasaka. The thugs injured Johnny, kidnapped Alt and took off in a van. Arasaka wanted her to develop a new variant of the Soulkiller software she had written for ITS.[13]
In 2013, Rogue and her partner, Nomad Santiago were mercs, being

### To add text to existing vector database 
#### (Optional, only if you want to add something that was not previously there in the above text file)

In [ ]:
# from langchain.embeddings import CohereEmbeddings
# import json
# import random
# from langchain.vectorstores import Chroma

#selected_key = json.load(open('apikeys.json', 'r'))['api_keys'][random.randint(
#    0, len(json.load(open('apikeys.json', 'r'))['api_keys'])-1)]
#embeddings = CohereEmbeddings(cohere_api_key=selected_key)
#persist_directory = 'Cyberpunk_2077/characters/Jackie_Welles/vectordb'
#vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
#vectordb.add_texts(["pineapples are yummy", "tomatoes are evil"])
#vectordb.persist()

#### To delete vectordb entirely, delete the public_vectordb folder inside the game folder

In [ ]:
import shutil
# path to vectordb inside your character
folder_path = 'Some_game/characters/Some_character/vectordb'
shutil.rmtree(folder_path)